### Data Preprocessing

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

In [36]:
#data selection
df = pd.read_csv('data_raw/ratings_small.csv')
df = df.loc[df.rating>=4.0]
#get number of movies watched by each userId
df_n = df['userId'].value_counts().to_frame()
df_n.columns = ['ntotal']
df_n['userId'] = df_n.index
# print(df_n) eliminate the data <21
df = df.merge(df_n, on = 'userId')
df = df.loc[df.ntotal>=21]

#sort by rating and timestap
df = df.sort_values(['userId','rating'],ascending=[1,0])
df = df.sort_values(['userId','timestamp'],ascending=[1,1])

df = df[['userId', 'movieId']]
df['count'] = df.groupby('userId').cumcount()
# df

In [37]:
#split and save inputs and truth ground 
df_X = df[df['count']<10]
df_Y = df[df['count']>=10]
df_X = df_X[['userId', 'movieId']]
df_Y = df_Y[['userId', 'movieId']]

df_X.to_csv('data/users_history.csv')
df_Y.to_csv('data/ground_truth.csv')

### "Movie2Movie" Co-visitation Matrix - Rating Weighted

In [13]:
df = pd.read_csv('data_raw/ratings_small.csv').drop('timestamp', axis=1)
df = pd.merge(df, df, on ='userId')
df = df.loc[df.movieId_x != df.movieId_y]
df['wgt'] = df.rating_y / 5
df = df[['movieId_x','movieId_y','wgt']]
df = df.groupby(['movieId_x','movieId_y']).wgt.sum()
df = df.reset_index()
df = df.sort_values(['movieId_x','wgt'], ascending=[True, False])
df = df.reset_index(drop = True)
df['n'] = df.groupby('movieId_x').movieId_y.cumcount()
df = df.loc[df.n <20].drop('n', axis=1)
df.to_csv('data/Movie2Movie.csv')

### Rerank

In [38]:
def df_covisitation_to_dict(df):
    return df.groupby('movieId_x').movieId_y.apply(list).to_dict()

def df_user_to_dict(df):
    return df.groupby('userId').movieId.apply(list).to_dict()

#read
df_cov = pd.read_csv('data/Movie2Movie.csv')
df_X = pd.read_csv('data/users_history.csv')
df_y = pd.read_csv('data/ground_truth.csv')

cov_dic = df_covisitation_to_dict(df_cov)
X_dic = df_user_to_dict(df_X)
gt_dic = df_user_to_dict(df_y)

In [39]:
print(df_X.head())
print(df_y.head())
print(df_cov.head())

   Unnamed: 0  userId  movieId
0           9       2      150
1          33       2      590
2          34       2      592
3          17       2      296
4          10       2      153
   Unnamed: 0  userId  movieId
0          15       2      266
1           7       2       50
2           5       2       39
3          21       2      454
4          23       2      480
   Unnamed: 0  movieId_x  movieId_y    wgt
0           0          1        356  139.6
1           1          1        260  137.9
2           2          1        318  132.3
3           3          1        296  131.7
4           4          1        593  119.1
